In [ ]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras  # Upgrade to Keras 3.

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras

# Use mixed precision to speed up all training in this guide.
keras.mixed_precision.set_global_policy("mixed_float16")

## Retriving Sentiment Data and Preprocessing

In [ ]:
import pandas as pd

# Load the dataset
dataset_path = '/content/Sentiment_NLP.csv'
df = pd.read_csv(dataset_path)

# Display the first few rows to understand its structure
print(df.head())

                                                text  sentiment
0  So there is no way for me to plug it in here i...          0
1                         Good case Excellent value.          1
2                             Great for the jawbone.          1
3  Tied to charger for conversations lasting more...          0
4                                  The mic is great.          1


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming the DataFrame has columns 'text' for the review text and 'label' for the sentiment
texts = df['text'].values
labels = df['sentiment'].values  # Ensure labels are numeric

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf

def make_dataset(data, labels, batch_size=16):
    dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    dataset = dataset.batch(batch_size)
    return dataset

# Create training and testing datasets
BATCH_SIZE = 16
train_dataset = make_dataset(X_train, y_train, BATCH_SIZE)
test_dataset = make_dataset(X_test, y_test, BATCH_SIZE)

In [ ]:
#Defining the preprocessor
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_large_en_uncased",
    sequence_length=512,
)

In [ ]:
#Using the preprocessor on train and test sets
train = (
    train_dataset.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
test = (
    test_dataset.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

In [ ]:
backbone = keras_nlp.models.BertBackbone.from_preset("bert_tiny_en_uncased")

backbone.trainable = False
inputs = backbone.input
sequence = backbone(inputs)["sequence_output"]
for _ in range(2):
    sequence = keras_nlp.layers.TransformerEncoder(
        num_heads=4,  # Adjusted number of heads
        intermediate_dim=256,  # Adjusted intermediate dimension
        dropout=0.2,  # Adjusted dropout rate
    )(sequence)

# Adding a dropout layer before the output layer for regularization
sequence = keras.layers.Dropout(0.5)(sequence[:, backbone.cls_token_index, :])
outputs = keras.layers.Dense(2)(sequence)

In [ ]:
model = keras.Model(inputs, outputs)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(1e-4),  # Adjusted optimizer and learning rate
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    jit_compile=True,  # Ensure it's supported by your TensorFlow version
)

In [ ]:
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer) │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ segment_ids (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_ids (InputLayer)    │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_backbone             │ [(None, 128), (None,   │      4,385,920 │ padding_mask[0][0],    │
│ (BertBackbone)            │ None, 128)]            │                │ segment_ids[0][0],     │
│                           │                        │                │ token_ids[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_4     │ (None, None, 128)      │        132,480 │ bert_backbone[0][1]    │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_5     │ (None, None, 128)      │        132,480 │ transformer_encoder_4… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_6 (GetItem)      │ (None, 128)            │              0 │ transformer_encoder_5… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_16 (Dropout)      │ (None, 128)            │              0 │ get_item_6[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 2)              │            258 │ dropout_16[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,651,138 (17.74 MB)

 Trainable params: 265,218 (1.01 MB)

 Non-trainable params: 4,385,920 (16.73 MB)

In [ ]:
# Training the model
model.fit(
    train,
    validation_data=test,
    epochs=6,
)

Epoch 1/6
100/100 ━━━━━━━━━━━━━━━━━━━━ 631s 6s/step - loss: 1.1358 - sparse_categorical_accuracy: 0.5074 - val_loss: 0.6148 - val_sparse_categorical_accuracy: 0.6466
Epoch 2/6
100/100 ━━━━━━━━━━━━━━━━━━━━ 592s 6s/step - loss: 0.8394 - sparse_categorical_accuracy: 0.5799 - val_loss: 0.5671 - val_sparse_categorical_accuracy: 0.6967
Epoch 3/6
100/100 ━━━━━━━━━━━━━━━━━━━━ 588s 6s/step - loss: 0.7152 - sparse_categorical_accuracy: 0.6286 - val_loss: 0.5496 - val_sparse_categorical_accuracy: 0.7093
Epoch 4/6
100/100 ━━━━━━━━━━━━━━━━━━━━ 546s 5s/step - loss: 0.7134 - sparse_categorical_accuracy: 0.6168 - val_loss: 0.5342 - val_sparse_categorical_accuracy: 0.7218
Epoch 5/6
100/100 ━━━━━━━━━━━━━━━━━━━━ 557s 6s/step - loss: 0.6524 - sparse_categorical_accuracy: 0.6526 - val_loss: 0.5247 - val_sparse_categorical_accuracy: 0.7168
Epoch 6/6
100/100 ━━━━━━━━━━━━━━━━━━━━ 590s 6s/step - loss: 0.6053 - sparse_categorical_accuracy: 0.6844 - val_loss: 0.5090 - val_sparse_categorical_accuracy: 0.7343
